In [ ]:
import pandas as pd               # to read in the synapse data
import numpy as np                # for numerical data
import matplotlib.pyplot as plt   # for plotting
import os                         # for OS specific operations such as directory/file manipulations
from collections import Counter
import import_ipynb as ipy
import csv
from matplotlib import cm
from annotationframeworkclient import FrameworkClient


"""
This is the format of creating a pd dataframe of specific synapses:

df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"pre_pt_root_id": neur array, "post_pt_root_id": neur_array})
"""

In [ ]:
datastack_name = "flywire_fafb_production"
client = FrameworkClient(datastack_name)

In [ ]:
client.annotation.get_tables()

versions = client.materialize.get_versions()
print(versions)

recent_version_info = client.materialize.get_version_metadata(versions[-1])
print(recent_version_info)

In [ ]:
#Reads xmin, xmax, ymin, ymax, zmin, zmax and converts them from pixels to nanometers

region_parameters = {}

region_file = pd.read_csv('') #Specify the file location

for i in range(len(region_file)):
    TempDict = {}
    
    for j in region_file:
        if j != "Region":
            if j == "zmin" or j == "zmax":
                TempDict[j] = int(region_file[j][i])*40
            else:
                TempDict[j] = int(region_file[j][i])*4
    
    CurrentReg = region_file["Region"][i]
    region_parameters[CurrentReg] = TempDict
    

    

# Functions

In [ ]:
def Filter_df(df, Region, Autapses = False, connection_score = 0, cleft_score = 0, min_dist = [120, 120, 120]):
    """
    Limits a df to various parameters.
    
    
    Parameters
    -------------
    df (pd df): A df based on synapse data given by client.materialize.query_table()
    Region (str): A region that is a key in the dict region_parameters
    Autapses (bool): False if you do not want autapses in the df (default)
    connection_score (int): The minimum connection score you want counted
    cleft_score (int): The minimum cleft score you want counted
    min_dist (list): The minimum distance between neurons in order to be considered different from each other.
    
    
    Returns
    -------------
    A pd df limited to the given region.
    
    
    """
    
    
    Lim_df = df
    
    
    RegDict = region_parameters[Region]
    Min_Values = [RegDict["xmin"], RegDict["ymin"], RegDict["zmin"]]
    Max_Values = [RegDict["xmax"], RegDict["ymax"], RegDict["zmax"]]
    
    Coord_Arrs = Lim_df["pre_pt_position"]
    Counter_Arr = np.zeros(len(Coord_Arrs), dtype = np.uint64)
    for i in range(len(Coord_Arrs)):
        for j in range(len(Coord_Arrs[i])):
            if Coord_Arrs[i][j] > Min_Values[j] and Coord_Arrs[i][j] < Max_Values[j]:
                Counter_Arr[i] += 1
    
    Lim_df["Temp_Arr"] = Counter_Arr
    Lim_df = Lim_df[Lim_df["Temp_Arr"] == 3]
    Lim_df.drop("Temp_Arr", axis = 1, inplace = True)
    
    
    if not Autapses:
        Lim_df = Lim_df[Lim_df["pre_pt_root_id"] != Lim_df["post_pt_root_id"]]
    
    
    Lim_df = Lim_df[Lim_df["connection_score"] >= connection_score]
    Lim_df = Lim_df[Lim_df["cleft_score"] >= cleft_score]
    
    
    
    
    Coords = Lim_df["pre_pt_position"]
    Pre_Partner = list(Lim_df["pre_pt_root_id"])
    Post_Partner = list(Lim_df["post_pt_root_id"])
    
    x_list = []
    y_list = []
    z_list = []
    coord_list = []

    for i in Coords:
        x_list.append(i[0])
        y_list.append(i[1])
        z_list.append(i[2])

    id_bools = [True] * len(x_list)
    
    bad_coords = []
    for i in range(len(x_list)):
        skip_loop = False
        for j in bad_coords:
            if i == j:
                skip_loop = True
                continue
        if skip_loop:
            continue
        for j in range(i+1, len(x_list)):
            if abs(x_list[i] - x_list[j]) < min_dist[0]:
                if abs(y_list[i] - y_list[j]) < min_dist[1]:
                    if abs(z_list[i] - z_list[j]) < min_dist[2]:
                        if Pre_Partner[i] == Pre_Partner[j]:
                            if Post_Partner[i] == Post_Partner[j]:
                                bad_coords.append(j)
       
    for i in bad_coords:
        id_bools[i] = False
    
    Lim_df = Lim_df[id_bools]
    
    
    
    
    return Lim_df
    
    
    

In [ ]:
def in_version(ID, Region = "connectome"): #Enter a neuron ID, returns True or False whether that ID is in the spreadsheet
    """
    To test whether a given ID is in the spreadsheet or not.
    
    
    Parameters
    -------------
    ID (str/int): An ID to test.
    
    
    Returns
    -------------
    False if not in region.
    True if in region.
    
    
    """
    
    pre_df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"post_pt_root_id": [ID]})
    post_df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"pre_pt_root_id": [ID]})
    
    if pre_df.empty and post_df.empty:
        return False
    else:
        return True

    
    

In [ ]:
def array_neurons(Neurons, Test = True):
    """
    Converts a long string of neurons into an array and prints which are not in the dataset.
    
    
    Parameters
    --------------
    Neurons (str): An inputted list of neuron IDs separated by commas.
    Test (bool): Tests whether the neurons are in the spreadsheet or not.
        If True: Removes all neurons that are not in the spreadsheet and informs you of them.
        If False: Keeps all neurons in the final array.
    
    
    Returns
    --------------
    The inputted neurons (that are in the dataset) in a numpy array.
    
    
    --------------
    """
    
    Neurons_array = np.array([x.strip() for x in Neurons.split(",")], dtype = np.uint64)
    
    if Test:
        not_in_spread = ""
        not_in_spread_index = []
        counter = 0

        for i in Neurons_array:
            if not in_version(i):
                if not_in_spread == "":
                    not_in_spread = str(i)
                else:
                    not_in_spread = not_in_spread + "," + str(i)
                not_in_spread_index.append(counter)
            counter += 1

        Neurons_array = np.delete(Neurons_array, not_in_spread_index)

        if not_in_spread_index:
            print("")
            print("These neurons are not in the dataset: ")
            print(not_in_spread)
            print("")
    
    return Neurons_array



In [ ]:
def dict_partners(Neurons, PrePost, Region, min_syn_partners = 1):
    """
    Returns a dict of all partners of a given neuron and how many synapses are between them.
    
    
    Parameters
    -------------
    Neurons (str): A list of neuron IDs.
    PrePost: "pre" or "post". The position in which partners you want.
        If "pre": Gives all presynaptic partners to ID. ID is in "post_pt_root_id".
        If "post": Gives all postsynaptic partners to ID. ID is in "pre_pt_root_id".    
    Region (str): A specified region.
    min_syn_partners (int): The minimum number of synaptic connections you want for a partner to be included.
    
    
    Returns
    -------------
    A dict.
        Keys (int): Neuron ID.
        Values (int): The number of synaptic connections between ID and the Key.
        
        
    Prerequisites
    -------------
    array_neurons (function)
    iterate_syn (function)
    LimRegion (function)
    region_parameters (dict)
    
    
    -------------
    """
    
    neur_array = array_neurons(Neurons)
        
    if PrePost == "pre":
        df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"post_pt_root_id": neur_array})
        df = Filter_df(df, Region)
        listed = list(df["pre_pt_root_id"])
    elif PrePost == "post":
        df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"pre_pt_root_id": neur_array})
        df = Filter_df(df, Region)
        listed = list(df["post_pt_root_id"])
    else:
        return 'Please enter "pre" or "post" for PrePost'
    
    listed_unique = list(set(listed))
    remove_neurons = []

    for i in listed_unique:
        if listed.count(i) < min_syn_partners:
            remove_neurons.append(i)
    
    for i in remove_neurons:
        while i in listed:
            listed.remove(i)
    
    ordered = Counter(listed)
    ordered = dict(sorted(ordered.items(), key = lambda x: x[1], reverse = True))
    
    return ordered



In [ ]:
def extract_connectome(PrePartners, PostPartners, Region):
    """
    Makes a numpy array of number of synapses between PrePartners and PostPartners.
    
    
    Parameters
    -------------
    PrePartners (str): All the presynaptic neurons, will create the rows, will go through array_neurons.
    PostPartners (str): All the postsynaptic neurons, will create the columns, will go through array_neurons.
    Region: The region the synapses are limited to.
    
    
    Returns
    -------------
    An array with the synapses between the PrePartners and PostPartners.
    
    
    Prerequisites
    -------------
    array_neurons (function)
    dict_partners (function)
    
    
    """
    
    
    Pre_Array = array_neurons(PrePartners)
    Post_Array = array_neurons(PostPartners)
    
    m = Pre_Array.size
    n = Post_Array.size
    
    syn_map = np.zeros((m,n), dtype = np.uint64)
    print("Number of presynaptic neurons: ", m)
    print("Number of postsynaptic neurons: ", n)
    print("========================================")

    for i in range(0, m):
        Current_Pre_Partners = dict_partners(str(Pre_Array[i]), "post", Region)
        
        for j in range(0,n):
            if Post_Array[j] in Current_Pre_Partners:
                syn_map[i,j] = Current_Pre_Partners[Post_Array[j]]
    
    return syn_map
    
    

In [ ]:
def plot_connectivity(pre_keys, post_keys, Region, Graph_Name = "Synaptic Connections", binarizing_threshold=5):
    """
    Plots the number of synapses between pre and post neurons.
    
    
    Parameters
    --------------
    pre_keys (np array): An array in which each value is a string which is a key in syn_dict_dnu. Will be the types of neurons analyzed as presynaptic.
    post_keys (np array): An array in which each value is a string which is a key in syn_dict_dnu. Will be the types of neurons analyzed as postsynaptic.
    Region (str): The region to which the plot will be limited.
    Graph_Name (str): The title of the graph.
    binarizing_threshold (int): Not 100% sure.
    
    
    Returns
    --------------
    A figure plotting the number of synapses between pre and post neurons.
    
    
    
    Prerequisites
    --------------
    array_neurons (function)
    extract_connectome (function)
    
    
    """
    pre_ns = np.zeros(pre_keys.size, dtype = int)
    pre_label_position = np.zeros(2*pre_keys.size, dtype = float)
    pre_neurons = ""
    pre_label_list = np.array([], dtype = str)
    
    for i in range(pre_keys.size):
        current_type = pre_keys[i]
        current_neur_array = array_neurons(syn_dict_dnu[current_type])
        pre_ns[i] = current_neur_array.size
        pre_label_position[2*i] = sum(pre_ns) - pre_ns[i] - 0.5
        pre_label_position[2*i + 1] = sum(pre_ns) - pre_ns[i]/2 - 0.5
        if i == 0:
            pre_neurons = pre_neurons + syn_dict_dnu[current_type]
        else:
            pre_neurons = pre_neurons + "," + syn_dict_dnu[current_type]
        pre_label_list = np.concatenate([pre_label_list,['–––––––––––', current_type]])
    pre_label_position = np.concatenate([pre_label_position, [sum(pre_ns) - 0.5]])
    pre_label_list = np.concatenate([pre_label_list, ['–––––––––––']])
    
    post_ns = np.zeros(post_keys.size, dtype = int)
    post_label_position = np.zeros(2*post_keys.size, dtype = float)
    post_neurons = ""
    post_label_list = np.array([], dtype = str)
    
    for i in range(post_keys.size):
        current_type = post_keys[i]
        current_neur_array = array_neurons(syn_dict_dnu[current_type])
        post_ns[i] = current_neur_array.size
        post_label_position[2*i] = sum(post_ns) - post_ns[i] - 0.5
        post_label_position[2*i + 1] = sum(post_ns) - post_ns[i]/2 - 0.5
        if i == 0:
            post_neurons = post_neurons + syn_dict_dnu[current_type]
        else:
            post_neurons = post_neurons + "," + syn_dict_dnu[current_type]        
        post_label_list = np.concatenate([post_label_list,['–––––––––––', current_type]])
    post_label_position = np.concatenate([post_label_position, [sum(post_ns) - 0.5]])
    post_label_list = np.concatenate([post_label_list, ['–––––––––––']])
    
    synapses_array = extract_connectome(pre_neurons, post_neurons, Region)
    
    for p in range(2):
        separator = ', ' 
        tstr = separator.join(pre_keys) #joins the type names with comma and space
        
        # If the title string is too long, then the file name may generate an error
        if len(tstr)>100:
            tstr = tstr[0:100]
        if p==0: # plot the original
            title_str = tstr
        else:    # plot the binarized version
            synapses_array[synapses_array<binarizing_threshold] = 0
            synapses_array[synapses_array>=binarizing_threshold] = 1
            title_str = tstr + ' (binarized)'

        # Draw a figure
        figA = plt.figure(figsize = (15,15))
        ax1 = figA.add_subplot(1,1,1)
        img = ax1.imshow(synapses_array, cmap = plt.get_cmap("Purples"), interpolation='none')
        figA.colorbar(img, ax=ax1)

        ax1.set_title(Graph_Name)

        ax1.set_xlabel('Post-Synaptic Neurons')
        ax1.set_xticks(post_label_position)
        ax1.set_xticklabels(post_label_list,rotation=90)
        #ax1.set_xticklabels(label_list)

        ax1.set_ylabel('Pre-Synaptic Neurons')
        ax1.set_yticks(pre_label_position)
        ax1.set_yticklabels(pre_label_list)

        plt.show()

        figA.savefig(
                os.path.join('figures', title_str + '.png'), format='png',  # PNG plot keeps the resolution but the font is distorted. PDF is opposite.
                bbox_inches='tight',
                #transparent=True,
                pad_inches=0.1,
                dpi=150 # For publication, use higher dpi
            ) 


    
    return figA
    
    
    


# Takes a while (only run if making connectivity charts).

In [ ]:
"""For importing neuron types from txt file"""


import time
start_time = time.time()


syn_dict_dnu = {}

with open('' , 'r', encoding = "utf8") as f: #Specify the file location
    
    lines = f.readlines()
    
    for i in range(len(lines)):
        
        if lines[i] == "Neuron Type:\n":
            types = [x.strip() for x in lines[i+1].split(",")]
            
            neurons = lines[i+2].strip()
            print("")
            print(types)
            if not neurons == "":
                array_neurons(neurons)
            
            for j in types:
                if not j in syn_dict_dnu:
                    syn_dict_dnu[j] = neurons
                else:
                    syn_dict_dnu[j] = syn_dict_dnu[j] + "," + neurons


print(syn_dict_dnu)

print("My program took", time.time() - start_time, "to run")




# Below are all of the different things you can do.


In [ ]:
"""
Enter a long list of neurons in and see which are in the spreadsheet and which aren't.
"""

All_Neurons = input("Please enter neuron(s): ")
All_Neurons = array_neurons(All_Neurons, Test = False)

in_spread = ""
not_in_spread = ""
count1 = 0
count2 = 0

for i in All_Neurons:
    if in_version(i):
        if in_spread == "":
            in_spread = str(i)
        else:
            in_spread = in_spread + "," + str(i)
        count1 += 1
    else:
        if not_in_spread == "":
            not_in_spread = str(i)
        else:
            not_in_spread = not_in_spread + "," + str(i)
        count2 += 1


print("")
print("These neurons are in the spreadsheet: ")
print(in_spread)
print("There are " + str(count1) + " neurons.")
print("")
print("These neurons are not in the spreadsheet: ")
print(not_in_spread)
print("There are " + str(count1 + count2) + " total neurons.")




In [ ]:
"""
Enter neuron(s) and find their partners.
"""

neurs = input("Please enter neurons: ")
Region = input("What region? ")
PrePost = input("Would you like pre or post partners? ")

partners = dict_partners(neurs, PrePost, Region, min_syn_partners = 5)
print("")
print(partners)
print("")    
print(list(partners.keys()))




In [ ]:
"""
Make a synaptic map and export it to excel.
"""

Pre = input("Pre partners: ")
Post = input("Post partners: ")
Region = input("Please enter the region: ")

excel_maybe = input("Export to excel? (yes or no): ")
if excel_maybe == "yes":
    name = input("Name of File: ")

Map = extract_connectome(Pre, Post, Region)
print(Map)

if excel_maybe == "yes":
    excel = pd.DataFrame(data = Map)
    out_path = '' + name + '.xlsx' #Specify the desired file path
    excel.to_excel(out_path)




In [ ]:
"""
Make a synaptic plot.
"""

Pre_Types = input("Pre partner types: ")
Post_Types = input("Post partner types: ")
Pre = np.array([x.strip() for x in Pre_Types.split(",")], dtype = str)
Post = np.array([x.strip() for x in Post_Types.split(",")], dtype = str)

Region = input("Please enter the region: ")

Map = plot_connectivity(Pre, Post, Region)
print(Map)




In [ ]:
"""
Ranks the likelihoods of neurotransmitters used by a neuron.
"""
ID = input("Please enter neuron: ")
syn_df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"pre_pt_root_id": [ID]})

Region = input("Please enter region: ")
syn_df = Filter_df(syn_df, Region)

nt_dict = {}

NTs = ["gaba", "ach", "glut", "oct", "ser", "da"]

for i in NTs:
    nt_dict[i] = sum(list(syn_df[i]))/len(syn_df[i])

ordered = sorted(nt_dict.items(), key = lambda x: x[1], reverse = True)
print(ordered)




In [ ]:
"""
Makes a csv file of all of the synapses between presynaptic and postsynaptic partners.
"""
Pre = input("Please enter presynaptic partner(s): ")
Post = input("Please enter postsynaptic partner(s): ")
Region = input("Please enter region: ")
name = input("Please create a file name: ")

print("Pre: ")
Pre = array_neurons(Pre)
print("")
print("Post:")
Post = array_neurons(Post)
print("")

df = client.materialize.query_table("synapses_nt_v1", filter_in_dict = {"pre_pt_root_id": Pre, "post_pt_root_id": Post})
df = Filter_df(df, Region)

Temp_Coords = list(df["pre_pt_position"])
Coords = []
for i in Temp_Coords:
    Coords.append("(" + str(int(i[0]/4)) + ", " + str(int(i[1]/4)) + ", " + str(int(i[2]/40)) + ")")
    
count = list(range(1,len(Coords)+1))


out_path = '' + name + '.csv' #Specify the desired file path
head = ["Coordinate 1", "Coordinate 2", "Ellipsoid Dimensions", "Tags", "Description", "Segments IDs", "Parent ID", "Type", "ID"]


with open(out_path, "w", newline = "") as file:
    thewriter = csv.writer(file)

    thewriter.writerow(head)

    count = 1
    for j in range(len(Coords)):
        Rows = [Coords[j], "", "", "", count, "", "", "Point", ""]
        thewriter.writerow(Rows)

        count+=1
        




In [ ]:
"""
Reorders a set of neurons.
"""

order = []

old_neurs = []

new_neurs = [old_neurs[i] for i in order]

print(new_neurs)


